In [3]:
from keras.utils.data_utils import get_file
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import LambdaCallback
import spacy
import numpy as np
import sys
import io

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [4]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.4MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=5dac46ba6604e0181cb10afa85e548d6a9101ffad5eb0bc34d2d4f3c07ef734f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ttxr4tcp/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
import en_core_web_md
nlp = en_core_web_md.load()


In [0]:
nlp.max_length = 16275513

In [9]:
from google.colab import files
uploaded = files.upload()

Saving glove.6B.50d.txt to glove.6B.50d (1).txt


In [10]:
from google.colab import files
uploaded = files.upload()

Saving text.txt to text.txt


In [4]:
embeddings = {}
f = open('glove.6B.50d (1).txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings[word] = coefs
f.close()

print(f'Found {len(embeddings)} word vectors.')

Found 400000 word vectors.


In [7]:
embeddings['president'] 

array([-0.11875  ,  0.6722   ,  0.19444  ,  0.55269  ,  0.53698  ,
       -0.37237  , -0.73494  , -0.30575  , -0.92601  , -0.43276  ,
        0.026956 ,  0.66861  , -0.79097  , -0.015932 ,  0.53918  ,
        0.30341  , -0.67042  ,  0.0051129,  0.62272  , -0.55823  ,
       -0.10887  ,  0.57305  , -0.016149 , -1.1889   , -0.24318  ,
       -2.6289   ,  0.41262  , -0.12904  , -1.3238   ,  0.64731  ,
        2.3595   ,  0.34048  , -1.9889   , -0.79084  , -0.79739  ,
       -0.87998  , -0.72991  ,  0.011697 ,  0.090612 , -0.17287  ,
       -0.83274  ,  1.1932   , -0.75211  , -1.1603   , -0.10074  ,
        0.60224  , -1.3739   ,  0.33674  , -0.31224  ,  0.097583 ],
      dtype=float32)

In [0]:
with open('text.txt', 'r') as f:
  text = f.read()

In [0]:
nlp.add_pipe(nlp.create_pipe('sentencizer'))
doc = nlp(text, disable=['ner', 'parser'])

In [0]:
sentences = [sent.string.strip() for sent in doc.sents]